In [1]:
import configparser
import trainlib.utils as utils
import trainlib.cuts as cuts
from trainlib.SimpleModel import SimpleModel
import inspect
from trainlib.ModelFactory import ModelFactory
from trainlib.ModelCollection import ModelCollection
from trainlib.PCAWhiteningPreprocessor import PCAWhiteningPreprocessor
from trainlib.config import TrainingConfig

Welcome to JupyROOT 6.10/09


Using TensorFlow backend.


In [2]:
cuts_translation = {"no_cut": cuts.no_cut, "mZZ_cut": cuts.mZZ_cut, "WHhadr_cut": cuts.WHhadr_cut, "ZHhadr_cut": cuts.ZHhadr_cut}
model_translation = {"SimpleModel": SimpleModel}

In [3]:
class IdentityDict(dict):
    def __missing__(self, key):
        return key

In [4]:
class IntegerDict(dict):
    def __missing__(self, key):
        return int(key)

In [5]:
no_dict = IdentityDict()
int_dict = IntegerDict()

In [6]:
def process_dict(rawstring, translation_dict):
    retdict = {}
    for source in rawstring.split(','):
        key, cmd = source.split(':')
        retdict[key.strip()] = translation_dict.get(cmd.strip())
        
    return retdict    

In [7]:
def process_list(rawstring, translation_dict):
    retlist = []
    for entry in rawstring.split(','):
        retlist.append(entry.strip())
        
    return retlist

In [8]:
def process_model(config, model):
    model_section = config['[' + model]
    
    model_name = model
    base_model = model_translation[model_section['base_model']]
    input_columns = process_list(model_section['input_columns'], no_dict)
    preprocessor_cuts = eval(model_section['preprocessor_cuts'])
    
    pre = PCAWhiteningPreprocessor(processed_columns = input_columns, cuts = preprocessor_cuts)
    
    mod = base_model(model_name, input_columns)
    mod.build()
    
    return mod, pre

In [29]:
def process_model_collection(config, section, weight_path):
    cur_sect = config[section]
    
    mcoll_name = section
    H1_stream = process_dict(cur_sect['H1_stream'], cuts_translation)
    H0_stream = process_dict(cur_sect['H0_stream'], cuts_translation)
    model_names = process_list(cur_sect['models'], no_dict)
    
    mcoll = ModelCollection(mcoll_name, H1_stream, H0_stream)

    for model in model_names:
        mod, pre = process_model(config, model)
        sett = TrainingConfig()
        mcoll.add_model(pre, mod, sett)

    if weight_path is not None:
        mcoll.load_weights(weight_path + mcoll_name)
        
    return mcoll

In [10]:
def assemble_lambda_string(func):
    func_string = str(inspect.getsourcelines(func)[0])
    func_string = func_string.strip("['\\n']").split(" = ")[1]

    return func_string

In [11]:
def assemble_dict(in_dict, translation_dict):
    inverse_translation = {val: key for key, val in translation_dict.iteritems()}
    
    out_string = ""
    
    for key, val in in_dict.iteritems():
        out_string += key + ": "
        out_string += inverse_translation.get(val) + ", "
    
    # cut away the last two (superfluous) characters
    return out_string[:-2]

In [12]:
def assemble_list(in_list):
    out_string = ""
    
    for entry in in_list:
        out_string += entry + ", "
        
    return out_string[:-2]

In [13]:
def make_config_model(config, model, preprocessor):    
    model_name = model.name
    section_name = '[' + model_name + ']'
    base_model = model.__class__.__name__
    input_columns = assemble_list(model.input_columns)
    preprocessor_cuts = assemble_lambda_string(preprocessor.cuts)
        
    # make a new section for the model in the config file
    config[section_name] = {}
    cur_sect = config[section_name]
    
    cur_sect['base_model'] = base_model
    cur_sect['input_columns'] = input_columns
    cur_sect['preprocessor_cuts'] = preprocessor_cuts

In [14]:
def make_config_model_collection(config, mcoll):
    mcoll_name = mcoll.name
    H1_stream = mcoll.H1_stream
    H0_stream = mcoll.H0_stream
    model_names = mcoll.model_dict.keys()
    #weightpath = training_dir
    
    # make a new section in the config file
    config[mcoll_name] = {}
    cur_sect = config[mcoll_name]
    
    cur_sect['H1_stream'] = assemble_dict(H1_stream, cuts_translation)
    cur_sect['H0_stream'] = assemble_dict(H0_stream, cuts_translation)
    cur_sect['models'] = assemble_list(model_names)
    
    models = mcoll.model_dict.values()
    preprocessors = mcoll.preprocessor_dict.values()
    for model, preprocessor in zip(models, preprocessors):
        make_config_model(config, model, preprocessor)

In [15]:
mcoll = ModelFactory.GenerateStandardModelCollections(SimpleModel, "/data_CMS/cms/wind/CJLST_NTuples/", "/data_CMS/cms/wind/test_training_100epochs/training/")

found the following models belonging to this collection:
D_VBF_ggH_2j_ML
D_VBF_ggH_1j_ML
now attempting to load model 'D_VBF_ggH_2j_ML' from file '/data_CMS/cms/wind/test_training_100epochs/training/D_VBF_ggH_ML/D_VBF_ggH_2j_ML/final.hdf5
now attempting to load preprocessor settings for 'D_VBF_ggH_2j_ML' from file '/data_CMS/cms/wind/test_training_100epochs/training/D_VBF_ggH_ML/D_VBF_ggH_2j_ML/preprocessor.pkl
now attempting to load model 'D_VBF_ggH_1j_ML' from file '/data_CMS/cms/wind/test_training_100epochs/training/D_VBF_ggH_ML/D_VBF_ggH_1j_ML/final.hdf5
now attempting to load preprocessor settings for 'D_VBF_ggH_1j_ML' from file '/data_CMS/cms/wind/test_training_100epochs/training/D_VBF_ggH_ML/D_VBF_ggH_1j_ML/preprocessor.pkl
found the following models belonging to this collection:
D_WHh_ggH_2j_ML
now attempting to load model 'D_WHh_ggH_2j_ML' from file '/data_CMS/cms/wind/test_training_100epochs/training/D_WHh_ggH_ML/D_WHh_ggH_2j_ML/final.hdf5
now attempting to load preprocessor 

2018-03-23 16:07:46.286036: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-03-23 16:07:46.286062: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-03-23 16:07:46.286071: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.


In [16]:
config_outpath = "/home/llr/cms/wind/cmssw/CMSSW_9_4_2/src/ZZAnalysis/AnalysisStep/test/Python/outconfig.ini"

In [23]:
def ModelCollectionsToConfigFile(mcoll, configpath):
    config = configparser.ConfigParser()
    
    for model in mcoll:
        make_config_model_collection(config, model)
        
    with open(configpath, 'w') as configfile:
        config.write(configfile)

In [24]:
def ModelCollectionsFromConfigFile(configpath, weightpath):
    config = configparser.ConfigParser()
    config.read(configpath)
    
    raw_sections = config.sections()
    main_sections = [name for name in raw_sections if not name.startswith('[')]
    
    mcolls = []
    
    for section in main_sections:
        mcolls.append(process_model_collection(config, section, weightpath))
        
    return mcolls

In [25]:
ModelCollectionsToConfigFile(mcoll, config_outpath)

In [30]:
mcoll2 = ModelCollectionsFromConfigFile(config_outpath, "/data_CMS/cms/wind/test_training_100epochs/training/")

/cvmfs/cms.cern.ch/slc6_amd64_gcc630/external/py2-pippkgs/6.0-fmblme/lib/python2.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  This is separate from the ipykernel package so we can avoid doing imports until


found the following models belonging to this collection:
D_VBF_ggH_2j_ML
D_VBF_ggH_1j_ML
now attempting to load model 'D_VBF_ggH_2j_ML' from file '/data_CMS/cms/wind/test_training_100epochs/training/D_VBF_ggH_ML/D_VBF_ggH_2j_ML/final.hdf5
now attempting to load preprocessor settings for 'D_VBF_ggH_2j_ML' from file '/data_CMS/cms/wind/test_training_100epochs/training/D_VBF_ggH_ML/D_VBF_ggH_2j_ML/preprocessor.pkl
now attempting to load model 'D_VBF_ggH_1j_ML' from file '/data_CMS/cms/wind/test_training_100epochs/training/D_VBF_ggH_ML/D_VBF_ggH_1j_ML/final.hdf5
now attempting to load preprocessor settings for 'D_VBF_ggH_1j_ML' from file '/data_CMS/cms/wind/test_training_100epochs/training/D_VBF_ggH_ML/D_VBF_ggH_1j_ML/preprocessor.pkl
found the following models belonging to this collection:
D_WHh_ggH_2j_ML
now attempting to load model 'D_WHh_ggH_2j_ML' from file '/data_CMS/cms/wind/test_training_100epochs/training/D_WHh_ggH_ML/D_WHh_ggH_2j_ML/final.hdf5
now attempting to load preprocessor 